In [6]:
from pathlib import Path
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import single_meteor_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [19]:
titles = [title.strip().lower()
          for title in open('../prepared_data/lstm/te_titles.txt', encoding='utf-8').readlines()]


Obtém predições e faz tokenização

In [20]:
model_predictions = {
    model: [line.strip().lower() for line in open(f'../predictions/{model}.txt').readlines()]
    for model in ['lstm', 'bert_cls', 'bert_mask']
}

In [9]:
stemmer = SnowballStemmer('portuguese')

In [10]:
avg_meteor_score = {
    model: np.mean([single_meteor_score(titles[i], pred, stemmer=stemmer)
                    for i, pred in enumerate(predictions)])
    for model, predictions in model_predictions.items()
}

In [11]:
avg_meteor_score

{'lstm': 0.10382020078018833,
 'bert_cls': 0.10373584094684014,
 'bert_mask': 0.12589191821027562}

Faz tokenização

In [12]:
titles = [word_tokenize(title, language='portuguese') for title in titles]

In [13]:
model_predictions = {
    model: [word_tokenize(pred, language='portuguese') for pred in predictions]
    for model, predictions in model_predictions.items()
}

Calcula bleu scores

In [14]:
def cumulative_bleu_scores(titles, predictions, ngram_order=3):
    results = np.zeros(ngram_order)
    
    # Obtém título e predição para o texto `text_nb`.
    titles = [[title] for title in titles]
        
    for ngram in range(ngram_order):
        # Obtém o simplex de dimensão `ngram` e concatena o restante do vetor com zeros
        # p/ que este tenha dimensão 4.
        weighting = tuple(1/(ngram+1) if i <= ngram else 0 for i in range(ngram_order))
        results[ngram] = corpus_bleu(titles, predictions, weights=weighting,
                                              smoothing_function=SmoothingFunction().method3)

    
    return results

In [21]:
tokenized_titles = [word_tokenize(title, language='portuguese') for title in titles]
tokenized_model_predictions = {
    model: [word_tokenize(pred, language='portuguese') for pred in predictions]
    for model, predictions in model_predictions.items()
}

In [23]:
bleu_scores = {
        model: cumulative_bleu_scores(tokenized_titles, predictions, ngram_order=3)
        for model, predictions in tokenized_model_predictions.items()
}

In [25]:
titles[0]

'maravilha'

In [24]:
bleu_scores

{'lstm': array([0.15769991, 0.09350181, 0.06126979]),
 'bert_cls': array([0.17067533, 0.10432747, 0.06862665]),
 'bert_mask': array([0.17664902, 0.11164997, 0.07619126])}

----

In [15]:
bleu_scores = {
    model: cumulative_bleu_scores(    import ipdb; ipdb.set_trace()
, predictions, ngram_order=3)
    for model, predictions in model_predictions.items()
}

In [16]:
bleu_scores

{'lstm': array([0.15769991, 0.09350181, 0.06126979]),
 'bert_cls': array([0.17067533, 0.10432747, 0.06862665]),
 'bert_mask': array([0.17664902, 0.11164997, 0.07619126])}

In [130]:
bleu_scores

{'lstm': array([0.39483658, 0.28670587, 0.23307774]),
 'bert_cls': array([0.44140406, 0.31932965, 0.25769633]),
 'bert_mask': array([0.42321673, 0.31895299, 0.26614213])}

In [51]:
cumulative_bleu_scores(titles, model_predictions['lstm'])

array([0.15769991, 0.09350181, 0.06126979])

In [52]:
cumulative_bleu_scores(titles, model_predictions['bert_cls'])

array([0.17067533, 0.10432747, 0.06862665])

In [53]:
cumulative_bleu_scores(titles, model_predictions['bert_mask'])

array([0.17664902, 0.11164997, 0.07619126])

In [322]:
titles[3]

['fone', 'de', 'ouvido', 'péssima', 'qualidade']

Gera tabela NIST

In [316]:
reviews = [line.strip() for line in open('../prepared_data/lstm/te_reviews.txt', encoding='utf-8').readlines()]

In [329]:
model_predictions = {
    model: [line.strip() for line in open(f'../predictions/{model}.txt').readlines()]
    for model in ['lstm', 'bert_cls', 'bert_mask']
}

In [336]:
pd.DataFrame({
    'review': reviews[0:200],
    'modelo1': model_predictions['lstm'][0:200],
    'nota_modelo1': '',
    'modelo2': model_predictions['bert_cls'][0:200],
    'nota_modelo2': '',
    'modelo3': model_predictions['bert_mask'][0:200],
    'nota_modelo3': ''
}).to_excel('../scores/planilha_nota.xlsx')

## METEOR

In [16]:
import nltk

In [17]:
assert nltk.__version__ == '3.5'

In [39]:
wordnet.synsets('gato', lang='por')[0].lemma_names(lang='por')

['bichano',
 'gata',
 'gato',
 'gato-doméstico',
 'Gato_doméstico',
 'Gato-doméstico']

In [40]:
meteor_score('apenas um teste', 'apenas um testea', stemmer=stemmer)

NameError: name 'meteor_score' is not defined